# **Análisis exploratorio y estadística**
# AE15 - Introducción a métodos de clustering

<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="100" align="left" title="Runa-perth">
<br clear="left">
Contenido opcional

En este notebook  tenemos por objetivo introducir las técnicas de clustering.

## <font color='blue'>__Introducción__</font>

Clustering es una técnica de aprendizaje no supervisado en estadística y minería de datos que se utiliza para agrupar un conjunto de datos en subconjuntos o "clústeres" similares en función de características compartidas. En el contexto de una clase de estadística, los métodos de clustering buscan identificar patrones naturales en los datos sin requerir etiquetas predefinidas.

El objetivo principal de los métodos de clustering es maximizar la similitud intraclúster (los puntos dentro de un clúster son similares) y minimizar la similitud interclúster (los puntos de diferentes clústeres son diferentes).

Los métodos de clustering pueden ayudar a descubrir relaciones intrínsecas y segmentar datos en grupos significativos, lo que facilita la comprensión y el análisis de datos complejos.

Estos métodos son esenciales en la exploración y análisis de datos y se aplican en una amplia variedad de campos, desde la segmentación de clientes en negocios hasta la clasificación de genes en biología.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## <font color='blue'>**Leyendo los datos**</font>

El código está adaptado para poder leer desde google drive, sin embargo, se puede modificar la ruta si es que se quiere trabajar en forma local.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Becas_Capital_Humano_23/Material_clases_CD_AD_2023/M03-Analisis_exploratorio_estadistica/data/"
titanic_df = pd.read_csv(path + "titanic_train.csv").dropna()
titanic_df.head(3).T

Analice los siguientes gráficos. Observe como ciertas combinaciones de variables tienden a generar agrupamientos de puntos. Otras, sin embargo, se comportan de forma "lineal". Intente explicar estas situaciones desde el conocimiento que ya tiene de los datos.

In [ ]:
sns.set(context='notebook',style='darkgrid', palette='dark')
sns.pairplot(data=titanic_df, hue='Pclass', palette='viridis')
plt.show()

## <font color='blue'>**Clustering**</font>

Muchas veces la relación entre una variable dependiente y el conjunto de variables independiente no es evidente. Ya vimos en las visualizaciones de notebooks pasados (en la figura anterior hay una muestra de ellas) que la sobrevivencia de un pasajero se relacionaba con algunas de las variables del dataset. Pero podemos ir un poco mas allá en este análisis. ¿Podremos encontrar una relación no evidente (Multivariada) entre un conjunto de variables independiente y alguna variable que queramos modelar? como por ejemplo Survived. Para esto iremos un poco mas allá aplicando técnicas de clustering.

El __clustering__, también conocido como agrupamiento, es una técnica de aprendizaje no supervisado que tiene como objetivo dividir un conjunto de datos en grupos, de manera que los datos en el mismo grupo (llamado clúster) sean más similares entre sí que con los de otros grupos (clústeres). En otras palabras, el objetivo es separar los datos en grupos homogéneos basados en alguna medida de similitud (como la distancia euclidiana, la distancia de Manhattan, la similitud del coseno, etc.).

Algunos de los tipos más comunes de algoritmos de clustering son:

- __K-Means__: Es uno de los algoritmos de clustering más simples y populares. Selecciona k puntos iniciales (centroides) y asigna cada dato al centroide más cercano, luego recalcula los centroides como el promedio de todos los puntos en el clúster y repite el proceso hasta que los centroides ya no cambien significativamente.
<br>
<center>
<img src="https://drive.google.com/uc?export=view&id=1G9F8kjtHcaIEDVugEMnxJb9nu9gxg7SI" width='300'>
</center>

- __Clustering Jerárquico__: Este algoritmo crea una jerarquía de clústeres. Puede ser "Aglomerativo" (comienza con cada punto como un clúster y luego combina clústeres basados en alguna medida de similitud) o "Divisivo" (comienza con un solo clúster que contiene todos los puntos y luego lo divide recursivamente en clústeres más pequeños).
- __DBSCAN (Density-Based Spatial Clustering of Applications with Noise)__: Es un algoritmo basado en densidad que define los clústeres como áreas de alta densidad separadas por áreas de baja densidad. No requiere que se especifique el número de clústeres a priori y puede encontrar clústeres de forma arbitraria, lo que es una ventaja sobre K-Means.
<br>
<center>
<img src="https://drive.google.com/uc?export=view&id=1CgfBZwJM4IXYcO78L0t45tV6dekYB-ad" width='400'>
</center>

- __Expectation–Maximization (EM) Clustering usando Gaussian Mixture Models (GMM)__: Una de las principales desventajas de K-Means es su uso ingenuo del valor medio para el centro del clúster. Podemos ver por qué esto no es la mejor manera de hacer las cosas observando la imagen de abajo.
<br>
<center>
<img src="https://drive.google.com/uc?export=view&id=1FuQwp1sqQijNkvujeQ0YDej7q-T6PndN" >
</center>

    En el lado izquierdo, parece bastante obvio a simple vista que hay dos clústeres circulares con diferentes radios centrados en la misma media. K-Means no puede manejar esto porque los valores medios de los clústeres están muy próximos entre sí. K-Means también falla en casos donde los clústeres no son circulares, nuevamente como resultado de usar la media como centro del clúster.
    Los Modelos de Mezcla Gaussiana (GMM) nos ofrecen más flexibilidad que K-Means. Con GMM, asumimos que los puntos de datos se distribuyen de manera gaussiana; esta es una suposición menos restrictiva que decir que son circulares usando la media. De esa manera, tenemos dos parámetros para describir la forma de los clústeres: ¡la media y la desviación estándar! Tomando un ejemplo en dos dimensiones, esto significa que los clústeres pueden adoptar cualquier forma elíptica (ya que tenemos una desviación estándar en ambas direcciones, x e y). Así, cada distribución gaussiana se asigna a un único clúster.
    Para encontrar los parámetros de la gaussiana para cada clúster (por ejemplo, la media y la desviación estándar), utilizaremos un algoritmo de optimización llamado Expectation-Maximization (EM). Echa un vistazo al gráfico de abajo como ilustración de las gaussianas ajustándose a los clústeres. Luego podemos proceder con el proceso de clustering de Expectation-Maximization utilizando GMM.
    <br>
<center>
<img src="https://drive.google.com/uc?export=view&id=1FmKdiFgYxQ16S7UXOS3_0-_dX8UOSfsn" >
</center>


- __Clustering Espectral__: Utiliza los eigenvectores de una matriz de similitud para reducir la dimensionalidad de los datos antes de aplicar otro algoritmo de clustering (como K-Means). Es especialmente útil cuando la estructura de los clústeres es compleja y no necesariamente esférica.
- __Clustering basado en Modelos__: Este enfoque asume que los datos son generados por un conjunto de modelos internos. Uno de los ejemplos más conocidos es el "Gaussian Mixture Model" (GMM), que asume que los datos son generados por una mezcla de varias distribuciones gaussianas.
- __Clustering basado en Redes Neuronales__: Algunas redes neuronales, como el "Self-Organizing Map" (SOM) o el "Autoencoder", pueden ser utilizadas para realizar clustering. Estos modelos pueden capturar estructuras complejas y no lineales en los datos.
- __Clustering basado en Optimización__: En este enfoque, el problema del clustering se formula como un problema de optimización que se resuelve para encontrar la mejor división de los datos. Un ejemplo es el "Clustering basado en búsqueda de tabú".



## <font color='blue'>__K Means clustering__</font>

#### __La intuición__

K-Mans es un método de agrupamiento, que tiene como objetivo la partición de un conjunto de $N$ observaciones en $k$ grupos (o clusters) en el que cada observación pertenece al grupo cuyo valor medio es más cercano. Se suele usar la distancia cuadrática.

Tenemos un conjunto de datos de $N$ individuos, cada uno con un vector M-dimensional asociado que representa $M$ características. Cada entrada en los vectores representa una cantidad numérica diferente. Por ejemplo, cada vector podría representar un hogar individual, con atributos de ingreso, número de automóviles, número de hijos, etc.

El algoritmo consta de tres pasos:

1. __Inicialización:__ una vez escogido el número de grupos, k, se establecen k centroides en el espacio de los datos, por ejemplo, escogiéndolos aleatoriamente.
2. __Asignación objetos a los centroides:__ cada objeto de los datos es asignado a su centroide más cercano.
3. __Actualización centroides:__ se actualiza la posición del centroide de cada grupo tomando como nuevo centroide la posición del promedio de los objetos pertenecientes a dicho grupo.

Se repiten los pasos 2 y 3 hasta que los centroides no se mueven, o se mueven por debajo de una distancia umbral en cada paso.

![Partes](https://drive.google.com/uc?export=view&id=1vGANhhd8DrllEQIyLFdLBDEqTCSvtOKD)


## <font color='blue'>__Segmentación de clientes con K Means clustering__</font>

In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns

import plotly.express as pxp
import plotly.graph_objs as gph
import missingno as msno

from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN


In [ ]:
data = pd.read_csv(path + 'Customers.csv')
data.head()

In [ ]:
data.describe().T

In [ ]:
data.info()

In [ ]:
# o también
data.isnull().sum()

In [ ]:
msno.bar(data, fontsize=12)
plot.show()

In [ ]:
# Eliminamos columnas no requeridas
data.drop('CustomerID', axis=1, inplace=True)

### __EDA__

In [ ]:
sns.pairplot(data=data, hue ='Gender', palette='RdYlBu')
plt.show()

In [ ]:
plot.figure(figsize = (15, 4))
plotnum = 1

for cols in ['Age', 'AnnualIncome', 'SpendingScore']:
    if plotnum <= 3:
        ax = plot.subplot(1, 3, plotnum)
        sns.histplot(data[cols], ax=ax, kde=True, color='y')

    plotnum += 1

plot.tight_layout()
plot.show()

In [ ]:
# Definir los intervalos de edad
bins = [18, 25, 35, 45, 55, data['Age'].max()]
in_bins = [10, 30, 50, 70, 90, 110, data['AnnualIncome'].max()]

# Definir los nombres de los grupos
age_group_names = ['18-25', '26-35', '36-45', '46-55', '55+']
in_group_names = ['10-30', '31-50', '51-70', '71-90', '91-110', '111+']

# Crear una nueva columna en el DataFrame que representa el grupo de edad
data['AgeGroup'] = pd.cut(data['Age'], bins, labels=age_group_names, right=False)

# Crear una nueva columna en el DataFrame que representa el grupo de edad
data['IncomeGroup'] = pd.cut(data['AnnualIncome'], in_bins, labels=in_group_names, right=False)

# Ahora puede agrupar por la columna 'AgeGroup' y aplicar las operaciones que necesites
age_groups = data.groupby('AgeGroup')
age_groups.size()

In [ ]:
# Y por  agrupar por la columna 'IncomeGroup' y aplicar las operaciones que necesites
in_groups = data.groupby('IncomeGroup')
in_groups.size()

Ahora creemos un gráfico de barras para verificar la distribución de los clientes en grupos de edad particulares. También puedes aplicar lo mismo para visualizar el número de clientes versus los puntajes de gasto y el número de clientes según sus ingresos anuales.

In [ ]:
# Crear un gráfico de barras usando Seaborn
sns.countplot(x='AgeGroup', data=data, order=age_group_names)

# Añadir etiquetas y título
plt.xlabel('Grupo de Edad')
plt.ylabel('Conteo')
plt.title('Distribución de Edades')
# Mostrar el gráfico
plt.show()

In [ ]:
# Crear un gráfico de barras usando Seaborn
sns.countplot(x='IncomeGroup',
              data=data,
              order=in_group_names,
              palette='RdYlBu'
              )

# Añadir etiquetas y título
plt.xlabel('Grupo de Ingresos')
plt.ylabel('Conteo')
plt.title('Distribución de ingresos')
# Mostrar el gráfico
plt.show()

Veamos pares de variables: Ingreso Anual (AnnualIncome) y Puntuación de gasto (SpendingScore)

In [ ]:
# Crear un gráfico de dispersión usando Seaborn
sns.scatterplot(x='AnnualIncome', y='SpendingScore', data=data)

# Añadir etiquetas y título
plt.xlabel('Ingreso Anual')
plt.ylabel('Puntuación de Gasto')
plt.title('Relación entre Ingreso Anual y Puntuación de Gasto')

# Mostrar el gráfico
plt.show()

Comparemos con Edad

In [ ]:
# Crear un gráfico de dispersión usando Seaborn
sns.scatterplot(x='Age', y='SpendingScore',
                data=data)

# Añadir etiquetas y título
plt.xlabel('Edad')
plt.ylabel('Puntuación de Gasto')
plt.title('Relación entre Edad y Puntuación de Gasto')
plt.show()

Uno de los aspectos más críticos del clustering es seleccionar el valor correcto de $K$. Seleccionar $K$ al azar puede no ser una elección favorable. Utilizaremos el __método del codo__ y el __Coeficiente de Silueta__ para elegir el valor de $K$.

En nuestro caso, a partir del gráfico que se muestra a continuación, parece que el valor óptimo de $K$ encontrado mediante el método del codo es 4. Queremos maximizar el número de clústeres y limitar los casos en los que cada punto de datos se convierte en su propio centroide de clúster.

In [ ]:
x_input = data.loc[:, ['Age', 'SpendingScore']].values
wcss = []
for k in range(1, 12):
    k_means = KMeans(n_clusters=k, init='k-means++', n_init=10)
    k_means.fit(x_input)
    wcss.append(k_means.inertia_)

plot.figure(figsize=(8,4))

plot.plot(range(1, 12), wcss, linewidth=2, marker='8')
plot.title('Método del codo (elbow method)', fontsize=18)
plot.xlabel('K')
plot.ylabel('WCSS')
plot.show()

__WCSS__ significa __"Within-Cluster Sum of Squares"__ o __"Suma de Cuadrados Dentro del Clúster"__ en castellano. Es una métrica utilizada en técnicas de clustering, como el algoritmo K-Means, para evaluar la calidad de un agrupamiento de datos.

WCSS se refiere a la suma de las distancias al cuadrado entre cada punto de datos dentro de un clúster y el centroide de ese clúster. En otras palabras, mide cuán cerca están los puntos de datos dentro de un clúster de su centroide.

El objetivo del algoritmo K-Means es minimizar WCSS

### __Coeficiente de Silhouette__
Vamos a verificar cómo se ve el coeficiente de Silhouette para esta implementación en particular.

El coeficiente de Silhouette es una métrica utilizada para calcular la bondad de un agrupamiento, es decir, cuán similares son los objetos dentro del mismo clúster y cuán diferentes son los objetos en clústeres diferentes.

El valor del coeficiente de Silhouette varía entre -1 y 1, donde un valor alto indica que los objetos están bien agrupados, es decir, son similares entre sí en el mismo clúster y diferentes de los objetos en otros clústeres. Un valor cercano a 0 indica que los grupos se superponen, y un valor negativo sugiere que algunos objetos pueden haber sido agrupados incorrectamente.

En el contexto de la frase, se está sugiriendo que se va a examinar cómo se ve el coeficiente de Silhouette para una implementación específica de agrupamiento. Esto podría implicar evaluar la calidad del agrupamiento y posiblemente ajustar los parámetros del algoritmo de agrupamiento para mejorar el coeficiente de Silhouette.



Re entrenamos el modelo con *n_clusters=4*.

In [ ]:
k_means = KMeans(n_clusters=4, init='k-means++', n_init=10)
k_means.fit(x_input)

In [ ]:
from sklearn.metrics import silhouette_score
label = k_means.predict(x_input)
# Calculating Silhouette score
print(f' Silhouette Score(n=4): {silhouette_score(x_input, label):3.1f}')

In [ ]:
plot.figure(figsize = (10, 6))
plot.scatter(x_input[:, 0],
             x_input[:, 1],
             c=k_means.labels_*7,
             s=20)
plot.scatter(k_means.cluster_centers_[:, 0],
             k_means.cluster_centers_[:,1],
             color='red',
             s=50)
plot.title('Clusters de clientes', fontsize = 10)
plot.xlabel('Edad')
plot.ylabel('Puntuación de gasto (Spending Score)')
plot.show()

Del gráfico de arriba (Edad vs Puntuación de Gasto), se pueden observar los 4 clústeres que nos propone le modelo, con un __SC__ (Coeficiente de Silhouette) para de 0.5, lo cual es un mínimo aceptable.



<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="50" align="left" title="Runa-perth">
<br clear="left">
Fin contenido opcional